# 此notebook用来生成自定义bar，并进行分析

### 1. 导入必要的库

In [10]:
# 导入依赖并设置项目地址
import pandas as pd
from pathlib import Path
from typing import  Optional

from src.utils.logger import logger
from src.data_handler.bar_generator import BarGenerator
from src.data_handler.bar_rule import DollarBarRule, VolumeBarRule, TickBarRule

project_root = Path.cwd().parent.parent

### 2. 工具函数

In [11]:
def get_bar_rule(bar_type: str, threshold: float):
    """返回对应类型的 BarRule 实例。"""
    if bar_type == "dollar":
        return DollarBarRule(threshold=threshold)
    elif bar_type == "volume":
        return VolumeBarRule(threshold=threshold)
    elif bar_type == "tick":
        return TickBarRule(threshold=int(threshold))
    else:
        raise ValueError(f"Unknown bar type: {bar_type}")


def get_output_path(
    base_dir: Path,
    symbol: str,
    bar_type: str,
    period: str,
    period_key: Optional[str] = None,
) -> Path:
    """得到文件输出路径"""
    bars_dir = base_dir / f"{bar_type}_bars" / symbol

    if period == "none" or period_key is None:
        bars_dir.mkdir(parents=True, exist_ok=True)
        return bars_dir

    # 按周期层级组织目录
    if period == "day":
        year, month, _ = period_key.split("-")
        output_dir = bars_dir / year / month
    elif period == "month":
        year, _ = period_key.split("-")
        output_dir = bars_dir / year
    else:  # year
        output_dir = bars_dir

    output_dir.mkdir(parents=True, exist_ok=True)
    return output_dir


def format_threshold(bar_type: str, threshold: float) -> str:
    """格式化阈值（用于展示和文件名）。"""
    if bar_type == "dollar":
        if threshold >= 1_000_000:
            return f"{int(threshold / 1_000_000)}m"
        elif threshold >= 1_000:
            return f"{int(threshold / 1_000)}k"
        else:
            return str(int(threshold))
    elif bar_type == "volume":
        return str(int(threshold))
    else:  # tick
        return str(int(threshold))


def save_bars(
    df: pd.DataFrame,
    output_path: Path,
    symbol: str,
    bar_type: str,
    threshold: float,
    period_key: Optional[str] = None,
) -> Path:
    """保存 bar 数据到 CSV 文件。"""
    threshold_str = format_threshold(bar_type, threshold)

    if period_key:
        filename = f"{symbol}_{bar_type}_bars_{threshold_str}_{period_key}.csv"
    else:
        filename = f"{symbol}_{bar_type}_bars_{threshold_str}.csv"

    output_file = output_path / filename
    df.to_csv(output_file, index=False)
    return output_file


In [12]:
def run_generate_bar(
    bar_type: str,
    threshold: float,
    symbol: str = "BTCUSDT",
    start: str = "2025-01-01",
    end: Optional[str] = None,
    period: str = "none",
    output_dir: Optional[Path] = None,
    show_progress: bool = True,
):
    """Notebook 入口函数：从原始成交数据生成 bar。
    """

    # 归一化 period 参数
    period_norm = None if period == "none" else period

    # 输出目录
    output_base = output_dir or (project_root / "data" / "preprocess_data")
    output_base.mkdir(parents=True, exist_ok=True)

    logger.info(f"{'=' * 70}")
    logger.info("BAR GENERATION")
    logger.info(f"{'=' * 70}")
    logger.info(f"Bar Type: {bar_type.upper()}")
    logger.info(f"Symbol: {symbol}")
    logger.info(f"Threshold: {format_threshold(bar_type, threshold)}")
    logger.info(f"Period: {start} to {end or start}")
    logger.info(f"Grouping: {period_norm or 'single file'}")
    logger.info(f"Output: {output_base}")
    logger.info(f"{'=' * 70}")

    # 创建规则和生成器
    bar_rule = get_bar_rule(bar_type, threshold)
    bar_gen = BarGenerator()

    # 生成 bar
    result = bar_gen.generate(
        bar_rule=bar_rule,
        symbol=symbol,
        start_date=start,
        end_date=end,
        period=period_norm,
        show_progress=show_progress,
    )

    if period_norm is None:
        # 单个 DataFrame
        if result is None or result.empty:
            logger.info("⚠️  No bars generated!")
            return None

        output_path = get_output_path(output_base, symbol, bar_type, "none")
        output_file = save_bars(result, output_path, symbol, bar_type, threshold)

        logger.info(f"✅ Successfully saved {len(result)} bars to:")
        logger.info(f"{output_file}")

    else:
        # (period_key, DataFrame) 生成器
        for period_key, df_period in result:
            if df_period.empty:
                continue

            output_path = get_output_path(
                output_base,
                symbol,
                bar_type,
                period,
                period_key,
            )
            output_file = save_bars(
                df_period,
                output_path,
                symbol,
                bar_type,
                threshold,
                period_key,
            )

            logger.info(f"Saved {len(df_period)} bars to {output_file}")

### 3. 生成

In [13]:
# 按需修改参数后运行本单元即可生成 bar
run_generate_bar(
    bar_type="dollar",           # "dollar" / "volume" / "tick"
    threshold=560_000,
    symbol="BTCUSDT",
    start="2025-12-01",
    end="2025-12-31",
    period="none",               # "none" / "day" / "month" / "year"
    output_dir=project_root / "data" / "preprocess_data",
    show_progress=True,
)

| INFO     | __main__:run_generate_bar:21 - ======================================================================
| INFO     | __main__:run_generate_bar:22 - BAR GENERATION
| INFO     | __main__:run_generate_bar:23 - ======================================================================
| INFO     | __main__:run_generate_bar:24 - Bar Type: DOLLAR
| INFO     | __main__:run_generate_bar:25 - Symbol: BTCUSDT
| INFO     | __main__:run_generate_bar:26 - Threshold: 560k
| INFO     | __main__:run_generate_bar:27 - Period: 2025-12-01 to 2025-12-31
| INFO     | __main__:run_generate_bar:28 - Grouping: single file
| INFO     | __main__:run_generate_bar:29 - Output: e:\B\Economic&Finance\code\CryptoQuant\data\preprocess_data
| INFO     | __main__:run_generate_bar:30 - ======================================================================
| INFO     | src.data_handler.bar_generator:_generate_all:135 - 📊 Generating Bars: BTCUSDT | 2025-12-01 to 2025-12-31


Processing dates |██████████| 31/31 [01:24<00:00]

| INFO     | src.data_handler.bar_generator:_generate_all:169 - ✅ Complete: 8539 bars
| INFO     | __main__:run_generate_bar:55 - ✅ Successfully saved 8539 bars to:
| INFO     | __main__:run_generate_bar:56 - e:\B\Economic&Finance\code\CryptoQuant\data\preprocess_data\dollar_bars\BTCUSDT\BTCUSDT_dollar_bars_560k.csv
